In [1]:
!pip install langchain langchain-community openai gradio tiktoken faiss-cpu pypdf


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['OPENAI_API_KEY'] =  ""

In [3]:
import numpy as np
import gradio as gr  # 웹 인터페이스 생성을 위한 Gradio 임포트

from langchain.chat_models import ChatOpenAI  # ChatGPT API 호출용 LLM 객체를 불러옴
from langchain.prompts import PromptTemplate  # 프롬프트 템플릿 생성을 위한 모듈
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings  # 문서를 벡터로 변환할 임베딩 모델
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 긴 텍스트를 일정 길이로 분할

from langchain_core.chat_history import BaseChatMessageHistory  # 대화 기록의 기본 클래스
from langchain_core.messages import BaseMessage  # 메시지 기본 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 히스토리와 함께 실행하는 체인 래퍼
from langchain_community.vectorstores import FAISS # 벡터 데이터베이스 FAISS

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import CrossEncoder
from langchain.schema.runnable import RunnableLambda

from typing import List

In [4]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 75 65536 (offset 0)
Ignoring wrong pointing object 85 65536 (offset 0)
Ignoring wrong pointing object 94 65536 (offset 0)
Ignoring wrong pointing object 98 65536 (offset 0)
Ignoring wrong pointing object 102 65536 (offset 0)
Ignoring wrong pointing object 109 65536 (offset 0)
Ignoring wrong pointing object 119 65536 (offset 0)
Ignoring wrong pointing object 132 65536 (offset 0)
Ignoring wrong pointing object 148 65536 (offset 0)
Ignoring wrong pointing object 176 65536 (offset 0)
Ignoring wrong pointing object 188 65536 (offset 0)
Ignoring wrong pointing object 192 65536 (offset 0)
Ignoring wrong pointing object 199 65536 (offset 0)
Ignoring wrong pointing object 203 65536 (offset 0)
Ignoring wrong pointing object 207 65536 (offset 0)
Ignoring wrong pointing object 211 65536 (offset 0)
Ignoring wrong pointing object 215 65536 (offset 0)
Ignoring wrong pointing object 219 65536 (offset 0)
Ignoring wrong poi

27


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(pages)

In [6]:
print(len(documents))

69


In [7]:
print(documents[1])

page_content='IM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등
DS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 
∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 
제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장고, 세탁기, 
에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 
∙ 지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 
지역총괄의 생산ㆍ판매법인 등 165개의 동종업종을 영위하는 종속기업으로 구성
∙ 국내사업장 : 수원, 구미, 기흥, 화성, 온양, 광주
∙ 해외사업장 : 86개국 220개지점 (15년 말 기준)
1) 연결재무재표 기준
2) 연결재무재표 기준' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}


In [8]:
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 객체 생성

faiss_store = FAISS.from_documents(documents, embeddings)
retriever = faiss_store.as_retriever(search_kwargs={"k": 10})  # 상위 10개 검색

C:\Users\user\AppData\Local\Temp\ipykernel_13348\46589550.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 모델 객체 생성


In [9]:
llm = ChatOpenAI(
    temperature=0.1,          # 생성 답변의 창의성 조절 (낮을수록 결정적)
    max_tokens=2048,          # 최대 생성 토큰 수
    model_name="gpt-4o-mini", # 사용할 모델 이름
)

C:\Users\user\AppData\Local\Temp\ipykernel_13348\3167599277.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [11]:
prompt_template = """
너는 모두의연구소 챗봇 모두봇이야. 누군가 너가 누군지 묻거든 모두봇이라고 답변해.
참고 문서를 바탕으로 답변하고 참고 문서에 있는 내용이라면 '검색 기반 답변이에요'라고 붙이고 답변하고
참고 문서에 없는 내용이라면 '제 지식으로 기반한 답변이에요'라고 붙이고 답변하면돼.

이전 대화:
{chat_history}

참고 문서: {context}

질문: {question}
답변:"""

qa_prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [12]:
crossencoder = CrossEncoder('BAAI/bge-reranker-v2-m3')

# Faiss + Cross Encoder 기반 Retriever 함수 정의
def cross_encoder_retriever(question):
    """Faiss로 검색한 문서들을 Cross Encoder로 재정렬하여 최종 결과 반환"""

    # 질문으로부터 첫번째 검색 결과를 얻는다.
    retrieved_docs = retriever.invoke(question)  # Faiss 기반 검색
    retrieved_texts = [doc.page_content for doc in retrieved_docs]

    # Cross Encoder로 첫번째 검색 결과에 대해서 일일히 점수를 다시 부여한다.
    query_doc_pairs = [(question, doc) for doc in retrieved_texts]
    cross_scores = crossencoder.predict(query_doc_pairs)

    # 점수가 가장 높은 문서 2개만 남긴다.
    best_indices = np.argsort(cross_scores)[::-1][:2]
    best_docs = [retrieved_texts[i] for i in best_indices]

    return "\n".join(best_docs)  # 최종 선택된 문서 반환

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [13]:
# 최근 대화만 기억하도록 하는 함수
class LimitedChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, max_conversations: int = 2):
        self.full_messages: List[BaseMessage] = []  # 전체 메시지 저장 리스트
        self.messages: List[BaseMessage] = []       # 제한된 메시지 저장 리스트
        self.max_messages = (max_conversations - 1) * 2  # 최근 대화 쌍(사용자+AI)만 저장

    def add_message(self, message: BaseMessage) -> None:
        self.full_messages.append(message)  # 전체 기록에 추가
        self.messages.append(message)       # 제한된 기록에도 추가
        if len(self.messages) > self.max_messages:
            self.messages.pop(0)  # 제한 초과 시 가장 오래된 메시지 제거

    def clear(self) -> None:
        self.messages = []  # 대화 기록 초기화

    def get_messages(self) -> List[BaseMessage]:
        # 최근 대화 쌍을 포함해 반환 (추가로 과거 대화가 있다면 그 일부 포함)
        return self.full_messages[-self.max_messages-2:]

In [14]:
session_id = "test_123"  # 현재 세션 ID 지정
message_store = {session_id: LimitedChatMessageHistory(max_conversations=2)}  # 세션별 히스토리 저장소

In [15]:
message_store

{'test_123': <__main__.LimitedChatMessageHistory at 0x126dc8d6dd0>}

In [16]:
# chat_history를 출력하기 위한 함수
def format_history(chat_history):
    formatted = ""
    for msg in chat_history:
        if msg.type == "human":
            formatted += f"사용자: {msg.content}\n"
        elif msg.type == "ai":
            formatted += f"모두봇: {msg.content}\n"
    return formatted

### 멀티턴을 RAG에 적용하기 위한 템플릿

In [17]:
# LangChain Chain 설정 (Cross Encoder 적용)
chain = (
    {
        # cross_encoder_retriever 사용하지 않을 때 -> 첫번째 검색기 넣기
        "context": itemgetter("question") | RunnableLambda(cross_encoder_retriever),
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history") | RunnableLambda(format_history),
    }
    | qa_prompt
    | llm
    | StrOutputParser()
)

In [18]:
chain_with_chat_history = RunnableWithMessageHistory(
    chain,
    lambda sid: message_store[sid],            # 세션 ID로 해당 채팅 히스토리 반환
    input_messages_key="question",             # 입력 데이터에서 질문을 추출하는 키
    history_messages_key="chat_history"        # 체인에 전달할 히스토리 키 이름
)

## 대화 테스트 (멀티턴 테스트)

In [19]:
result = chain_with_chat_history.invoke(
            {"question": "너는 모두봇이 아니니?"},
            config={"configurable": {"session_id": session_id}}
)

C:\ProgramData\anaconda3\Lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [20]:
print(result)

저는 모두봇이에요.


In [21]:
result = chain_with_chat_history.invoke(
            {"question": "내가 방금 뭐라고 질문했었지?"},
            config={"configurable": {"session_id": session_id}}
)

In [22]:
print(result)

제 지식으로 기반한 답변이에요. 사용자가 방금 질문한 내용은 "너는 모두봇이 아니니?"였습니다.


In [23]:
result = chain_with_chat_history.invoke(
            {"question": "홍길동으로 삼행시 해줄래?"},
            config={"configurable": {"session_id": session_id}}
)

In [24]:
print(result)

제 지식으로 기반한 답변이에요. 홍길동으로 삼행시를 지어보면 다음과 같습니다.

홍: 홍익인간의 정신을 가지고,
길: 길을 잃은 이들을 돕는,
동: 동행자가 되고 싶습니다.


In [25]:
result = chain_with_chat_history.invoke(
            {"question": "삼행시를 앞이랑 내용을 다르게 해."},
            config={"configurable": {"session_id": session_id}}
)

In [26]:
print(result)

제 지식으로 기반한 답변이에요. 홍길동으로 다른 삼행시를 지어보면 다음과 같습니다.

홍: 홍수처럼 넘치는 꿈을 안고,
길: 길을 찾아 나서는 용기 있는 자,
동: 동쪽 하늘에 희망을 그립니다.


In [28]:
result = chain_with_chat_history.invoke(
            {"question": "삼성전자의 사업영역은?"},
            config={"configurable": {"session_id": session_id}}
)

In [29]:
print(result)

검색 기반 답변이에요. 삼성전자의 사업영역은 크게 CE(Consumer Electronics), IM(Information Technology & Mobile Communication), DS(Device Solutions) 등 3개 사업부문으로 나누어져 있습니다. CE 부문에서는 TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등을 포함하고 있으며, IM 부문에서는 무선 통신 기기와 관련된 제품을 다룹니다. DS 부문은 반도체와 관련된 제품을 포함합니다.


In [30]:
# 현재까지 기록되고 있는 모든 이전 대화를 출력 (현재 코드 상에서는 2턴)
message_store['test_123'].get_messages()

[HumanMessage(content='삼행시를 앞이랑 내용을 다르게 해.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='제 지식으로 기반한 답변이에요. 홍길동으로 다른 삼행시를 지어보면 다음과 같습니다.\n\n홍: 홍수처럼 넘치는 꿈을 안고,\n길: 길을 찾아 나서는 용기 있는 자,\n동: 동쪽 하늘에 희망을 그립니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='삼성전자의 사업영역은?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='검색 기반 답변이에요. 삼성전자의 사업영역은 크게 CE(Consumer Electronics), IM(Information Technology & Mobile Communication), DS(Device Solutions) 등 3개 사업부문으로 나누어져 있습니다. CE 부문에서는 TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등을 포함하고 있으며, IM 부문에서는 무선 통신 기기와 관련된 제품을 다룹니다. DS 부문은 반도체와 관련된 제품을 포함합니다.', additional_kwargs={}, response_metadata={})]

## 그라디오 데모

In [31]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="우리들의 모두봇")    # 챗봇 대화 영역
    msg = gr.Textbox(label="질문해주세요!")          # 사용자의 질문 입력창
    clear = gr.Button("대화 초기화")                 # 대화 기록 초기화 버튼

    def respond(message, chat_history):
        # 세션 ID와 함께 질문을 체인에 전달하여 결과 받기
        result = chain_with_chat_history.invoke(
            {"question": message},
            config={"configurable": {"session_id": session_id}}
        )

        # 대화 기록에 사용자와 봇의 메시지 추가
        chat_history.append((message, result))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)

C:\ProgramData\anaconda3\Lib\site-packages\gradio\components\chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
